In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date, time
import urllib
import json
import ssl
import folium
from folium import plugins
from folium.plugins import HeatMap
from haversine import haversine

---

### **Import the data**

Import CSV files

In [2]:
df_bonn = pd.read_csv("data/bonn.csv")
df_essen = pd.read_csv("data/essen.csv")

In [3]:
df_bonn.head()

,day,time,b_number,city,trip_duration,orig_lat,orig_lng,dest_lat,dest_lng
0,2019-03-06,01:13:00,21169,bonn,0 days 02:51:00.000000000,50.921682,6.959204,50.953793,6.899248
1,2019-03-06,14:43:00,21169,bonn,0 days 00:11:00.000000000,50.953793,6.899251,50.947463,6.922385
2,2019-03-06,15:21:00,21169,bonn,0 days 00:21:00.000000000,50.947462,6.922391,50.929703,6.932212
3,2019-03-06,17:45:00,21169,bonn,0 days 00:51:00.000000000,50.929703,6.932212,50.945799,6.915687
4,2019-03-07,05:58:00,21169,bonn,0 days 00:07:00.000000000,50.945765,6.915829,50.951143,6.915550


In [4]:
df_bonn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329629 entries, 0 to 329628
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   day            329629 non-null  object 
 1   time           329629 non-null  object 
 2   b_number       329629 non-null  int64  
 3   city           329629 non-null  object 
 4   trip_duration  329629 non-null  object 
 5   orig_lat       329629 non-null  float64
 6   orig_lng       329629 non-null  float64
 7   dest_lat       329629 non-null  float64
 8   dest_lng       329629 non-null  float64
dtypes: float64(4), int64(1), object(4)
memory usage: 22.6+ MB


Import API data

In [5]:
context = ssl._create_unverified_context()

def request(city_id):
    response = urllib.request.urlopen(f"https://api.nextbike.net/maps/nextbike-live.json?city={city_id}", context=context)
    response = response.read().decode("utf-8")
    return json.loads(response)

bonn_json = request(547)["countries"][0]["cities"][0] # Bonn city ID at NextBike is 547
essen_json = request(133)["countries"][0]["cities"][0] # Essen city ID at NextBike ist 133

In [6]:
# Use as center points for maps:
bonn_center = np.array([bonn_json["lat"], bonn_json["lng"]])
essen_center = np.array([essen_json["lat"], essen_json["lng"]])

In [7]:
# Use as city bounds for maps:
bonn_bounds = dict()
essen_bounds = dict()

# Bonn:
bonn_bounds["south_west"] = np.array([bonn_json["bounds"]["south_west"]["lat"], 
                                      bonn_json["bounds"]["south_west"]["lng"]])
bonn_bounds["north_east"] = np.array([bonn_json["bounds"]["north_east"]["lat"], 
                                      bonn_json["bounds"]["north_east"]["lng"]])
# Essen:
essen_bounds["south_west"] = np.array([essen_json["bounds"]["south_west"]["lat"], 
                                       essen_json["bounds"]["south_west"]["lng"]])
essen_bounds["north_east"] = np.array([essen_json["bounds"]["north_east"]["lat"], 
                                       essen_json["bounds"]["north_east"]["lng"]])

---

### **Format the data**

In [8]:
def format_trip_duration(df):
    """
    Assumption: all time durations are less than 1 day
    Assumption: all time durations are measured in seconds (not milliseconds)
    First turn "0 days 02:51:00.0000" into list split by " ", so we get 
    x=["0", "days", "02:51:00.0000"].
    Since all trip_durations are less than 1 day, we can discard x[0] and x[1] and 
    focus on x[2].
    We then split x[2], i.e. "02:51:00.0000", into a list split by ":", so we get 
    x=["02", "51", "00.0000"].
    We now turn these elements into numbers and pass them into the timedelta constructor.
    Timedelta objects describe differences in time, like time durations.
    """
    df["trip_duration"] = df["trip_duration"].apply(
        lambda s: s.split(" ")[2].split(":"))
    df["trip_duration"] = df["trip_duration"].apply(
        lambda x: timedelta(hours=int(x[0]), minutes=int(x[1]), seconds=int(float(x[2]))))
    return df

def str_to_date(row):
    return datetime.strptime(f"{row['day']} {row['time']}", "%Y-%m-%d %H:%M:%S")

In [9]:
%%time
# Sort values, reset index (and drop old index)
# Bonn:
df_bonn.sort_values(["day", "time"], inplace=True)
df_bonn = df_bonn.reset_index()
df_bonn = df_bonn.drop("index", axis=1)
# Essen:
df_essen.sort_values(["day", "time"], inplace=True)
df_essen = df_essen.reset_index()
df_essen = df_essen.drop("index", axis=1)

# Format trip duration
df_bonn = format_trip_duration(df_bonn)
df_essen = format_trip_duration(df_essen)

# Create timestamp, weekday, hour columns
# Bonn:
df_bonn["timestamp"] = df_bonn.apply(str_to_date, axis=1)
df_bonn["weekday"] = df_bonn["timestamp"].apply(lambda row: row.weekday())
df_bonn["hour"] = df_bonn["timestamp"].apply(lambda row: row.hour)
# Essen:
df_essen["timestamp"] = df_essen.apply(str_to_date, axis=1)
df_essen["weekday"] = df_essen["timestamp"].apply(lambda row: row.weekday())
df_essen["hour"] = df_essen["timestamp"].apply(lambda row: row.hour)

# Combine longitude/latitude coordinate columns: orig
# Bonn:
df_bonn["orig"] = list(zip(df_bonn["orig_lat"].round(4), df_bonn["orig_lng"].round(4)))
df_bonn["orig"] = df_bonn["orig"].apply(np.array) # turn tuples in np.arrays
df_bonn["dest"] = list(zip(df_bonn["dest_lat"].round(4), df_bonn["dest_lng"].round(4)))
df_bonn["dest"] = df_bonn["dest"].apply(np.array) # turn tuples in np.arrays
# Essen:
df_essen["orig"] = list(zip(df_essen["orig_lat"].round(4), df_essen["orig_lng"].round(4)))
df_essen["orig"] = df_essen["orig"].apply(np.array) # turn tuples in np.arrays
df_essen["dest"] = list(zip(df_essen["dest_lat"].round(4), df_essen["dest_lng"].round(4)))
df_essen["dest"] = df_essen["dest"].apply(np.array) # turn tuples in np.arrays

CPU times: user 17.1 s, sys: 278 ms, total: 17.4 s
Wall time: 17.4 s


In [10]:
df_bonn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329629 entries, 0 to 329628
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype          
---  ------         --------------   -----          
 0   day            329629 non-null  object         
 1   time           329629 non-null  object         
 2   b_number       329629 non-null  int64          
 3   city           329629 non-null  object         
 4   trip_duration  329629 non-null  timedelta64[ns]
 5   orig_lat       329629 non-null  float64        
 6   orig_lng       329629 non-null  float64        
 7   dest_lat       329629 non-null  float64        
 8   dest_lng       329629 non-null  float64        
 9   timestamp      329629 non-null  datetime64[ns] 
 10  weekday        329629 non-null  int64          
 11  hour           329629 non-null  int64          
 12  orig           329629 non-null  object         
 13  dest           329629 non-null  object         
dtypes: datetime64[ns](1), float64(4), in

---

### **Are there outliers?**

We want to identify all trips whose origin and destination points are outside of the borders gained from the API.  
Bounds Bonn: ``"south_west":{"lat":50.6489,"lng":6.2762},"north_east":{"lat":50.9436,"lng":7.2696}``  
Bounds Essen: ``"south_west":{"lat":51.3614,"lng":6.762},"north_east":{"lat":51.5275,"lng":7.4147}`` 

#### Bonn:

In [11]:
# Get the position outliers
df_bonn_bounds = df_bonn[
    ((df_bonn["orig"].str[0] > bonn_bounds["north_east"][0]) | (df_bonn["orig"].str[0] < bonn_bounds["south_west"][0])) | 
    ((df_bonn["orig"].str[1] > bonn_bounds["north_east"][1]) | (df_bonn["orig"].str[1] < bonn_bounds["south_west"][1])) | 
    ((df_bonn["dest"].str[0] > bonn_bounds["north_east"][0]) | (df_bonn["dest"].str[0] < bonn_bounds["south_west"][0])) | 
    ((df_bonn["dest"].str[1] > bonn_bounds["north_east"][1]) | (df_bonn["dest"].str[1] < bonn_bounds["south_west"][1]))]

# Get all entries which are in the API bounds
df_bonn = df_bonn[~df_bonn.isin(df_bonn_bounds).all(1)]

In [12]:
# Bounds-Plot on Map

def zone_map(location, lower_left, upper_right):
        # Generate a map - centered on the given location
        if (location == bonn_center).all():
            location = location-np.array([-0.1, 0.3]) # shift by [-0.1, 0.3] to make it look better (more centered)
        elif (location == essen_center).all():
            location = location-np.array([0, 0]) # shift by [0.1, 0] to make it look better (more centered)
            
        zone_map = folium.Map(
            location = location,
            tiles = "OpenStreetMap",
            zoom_start = 10)
        
        # Two corners of the rectangle are known(lower_left and upper_right)
        # upper_left and lower_right have to be calculated
        # This can throw an exception, if lower_left and upper_right are not in the correct format
        upper_left = (upper_right[0], lower_left[1])
        lower_right = (lower_left[0], upper_right[1])
        
        # Set markers on the map
        folium.Marker(lower_left).add_to(zone_map)
        folium.Marker(upper_left).add_to(zone_map)
        folium.Marker(lower_right).add_to(zone_map)
        folium.Marker(upper_right).add_to(zone_map)
        
        # Create a rectangle
        folium.Rectangle(
            bounds = [lower_left, upper_left, lower_right, upper_right],
            color = " #5dade2 ",
            fill = True,
            fill_color = " #5dade2 ",
            fill_opacity = 0.15).add_to(zone_map)
        
        return zone_map

In [13]:
# Plot zone for Bonn
zone_map(bonn_center, bonn_bounds["south_west"], bonn_bounds["north_east"])

In [14]:
# Print the outliers (but only relevant columns)
df_bonn_bounds.drop(["day", "time", "orig_lat", "orig_lng", "dest_lat", "dest_lng"], axis=1)

,b_number,city,trip_duration,timestamp,weekday,hour,orig,dest
23182,45075,bonn,01:50:00,2019-02-24 16:45:00,6,16,"[50.6435, 7.2173]","[50.726, 7.1538]"
27514,45161,bonn,00:47:00,2019-02-27 16:24:00,2,16,"[50.7173, 7.1333]","[50.6432, 7.2067]"
27691,45161,bonn,00:31:00,2019-02-27 17:28:00,2,17,"[50.6432, 7.2067]","[50.6842, 7.1554]"
35241,21169,bonn,02:51:00,2019-03-06 01:13:00,2,1,"[50.9217, 6.9592]","[50.9538, 6.8992]"
35927,21169,bonn,00:11:00,2019-03-06 14:43:00,2,14,"[50.9538, 6.8993]","[50.9475, 6.9224]"
...,...,...,...,...,...,...,...,...
317980,44677,bonn,00:07:00,2019-06-27 15:22:00,3,15,"[50.7265, 7.0928]","[51.0319, 6.6671]"
327606,44586,bonn,00:21:00,2019-06-30 08:51:00,6,8,"[50.6635, 7.1751]","[50.6342, 7.207]"
327844,44586,bonn,00:19:00,2019-06-30 10:49:00,6,10,"[50.6342, 7.2072]","[50.6602, 7.1719]"
328214,45163,bonn,00:11:00,2019-06-30 13:29:00,6,13,"[50.7414, 7.0864]","[50.5746, 6.9871]"


In [15]:
# Print all entries which are in the API bounds (but only relevant columns)
df_bonn.drop(["day", "time", "orig_lat", "orig_lng", "dest_lat", "dest_lng"], axis=1)

,b_number,city,trip_duration,timestamp,weekday,hour,orig,dest
0,44894,bonn,00:07:00,2019-02-01 00:03:00,4,0,"[50.7366, 7.1009]","[50.739, 7.093]"
1,44973,bonn,00:20:00,2019-02-01 00:12:00,4,0,"[50.7364, 7.0927]","[50.732, 7.1004]"
2,44802,bonn,00:06:00,2019-02-01 00:37:00,4,0,"[50.7372, 7.0911]","[50.7336, 7.095]"
3,44717,bonn,00:07:00,2019-02-01 00:38:00,4,0,"[50.7476, 7.0826]","[50.7543, 7.0715]"
4,44639,bonn,00:10:00,2019-02-01 00:44:00,4,0,"[50.7392, 7.0906]","[50.7525, 7.0712]"
...,...,...,...,...,...,...,...,...
329624,44862,bonn,00:46:00,2019-06-30 23:49:00,6,23,"[50.7159, 7.1133]","[50.7253, 7.1535]"
329625,44838,bonn,00:06:00,2019-06-30 23:50:00,6,23,"[50.6495, 7.2006]","[50.6577, 7.1942]"
329626,45084,bonn,00:21:00,2019-06-30 23:54:00,6,23,"[50.7377, 7.1018]","[50.7096, 7.1155]"
329627,45010,bonn,00:05:00,2019-06-30 23:55:00,6,23,"[50.7374, 7.1033]","[50.7444, 7.1004]"


In [16]:
def outliers_zone_map(location, lower_left, upper_right, df_outliers):
        #First draw a map like zone_map() does
        outliers_zone_map = zone_map(location, lower_left, upper_right)
    
        # Now outliers have to be added to the map
        # The outliers are stored in df_outliers
        for index, row in df_outliers.iterrows():
            
            # Save startpoint and endpoint
            startpoint = tuple(row["orig"])
            endpoint = tuple(row["dest"])
            
            # Draw the lines between startpoints and endpoints
            folium.PolyLine(
            locations = [startpoint, endpoint],
            color = "purple",
            weight = 2,
            opacity = 0.5
            ).add_to(outliers_zone_map)
            
            # Draw Cricles for startpoints and endpoints
            # Green circles = startpoints
            folium.CircleMarker(
            location = startpoint,
            radius = 4,
            color = "green").add_to(outliers_zone_map)
            # Red circles = endpoints
            folium.CircleMarker(
            location = endpoint,
            radius = 4,
            color = "crimson").add_to(outliers_zone_map) 
        
        return outliers_zone_map

In [17]:
# Outliers Plot for Bonn
outliers_zone_map(bonn_center, bonn_bounds["south_west"], bonn_bounds["north_east"], df_bonn_bounds)

Furthermore we want to remove trips with a high speed value.  
Therefore we calculate the linear distance between origin and destination with the haversine formular.  
Afterwards the km/h for each trip gets computed and journeys with a speed of more then 25 km/h are removed. 

![BikeURL](https://media.giphy.com/media/ftvphb1LgYP9SgoNGn/giphy.gif "bike")

In [18]:
# Calculate the distance in kilometers between origin and destination
df_bonn["distance (km)"] = df_bonn.apply(lambda row: haversine(row["orig"], row["dest"]), axis=1)
df_bonn.drop(["day", "time", "orig_lat", "orig_lng", "dest_lat", "dest_lng"], axis=1)

,b_number,city,trip_duration,timestamp,weekday,hour,orig,dest,distance (km)
0,44894,bonn,00:07:00,2019-02-01 00:03:00,4,0,"[50.7366, 7.1009]","[50.739, 7.093]",0.616674
1,44973,bonn,00:20:00,2019-02-01 00:12:00,4,0,"[50.7364, 7.0927]","[50.732, 7.1004]",0.730093
2,44802,bonn,00:06:00,2019-02-01 00:37:00,4,0,"[50.7372, 7.0911]","[50.7336, 7.095]",0.485359
3,44717,bonn,00:07:00,2019-02-01 00:38:00,4,0,"[50.7476, 7.0826]","[50.7543, 7.0715]",1.079285
4,44639,bonn,00:10:00,2019-02-01 00:44:00,4,0,"[50.7392, 7.0906]","[50.7525, 7.0712]",2.012538
...,...,...,...,...,...,...,...,...,...
329624,44862,bonn,00:46:00,2019-06-30 23:49:00,6,23,"[50.7159, 7.1133]","[50.7253, 7.1535]",3.016851
329625,44838,bonn,00:06:00,2019-06-30 23:50:00,6,23,"[50.6495, 7.2006]","[50.6577, 7.1942]",1.017326
329626,45084,bonn,00:21:00,2019-06-30 23:54:00,6,23,"[50.7377, 7.1018]","[50.7096, 7.1155]",3.270024
329627,45010,bonn,00:05:00,2019-06-30 23:55:00,6,23,"[50.7374, 7.1033]","[50.7444, 7.1004]",0.804671


In [19]:
# Calculate the km/h for each trip
df_bonn["km/h"] = df_bonn.apply(lambda row: row["distance (km)"]/(row["trip_duration"].seconds/3600), axis=1)
df_bonn.drop(["day", "time", "orig_lat", "orig_lng", "dest_lat", "dest_lng"], axis=1)

,b_number,city,trip_duration,timestamp,weekday,hour,orig,dest,distance (km),km/h
0,44894,bonn,00:07:00,2019-02-01 00:03:00,4,0,"[50.7366, 7.1009]","[50.739, 7.093]",0.616674,5.285779
1,44973,bonn,00:20:00,2019-02-01 00:12:00,4,0,"[50.7364, 7.0927]","[50.732, 7.1004]",0.730093,2.190280
2,44802,bonn,00:06:00,2019-02-01 00:37:00,4,0,"[50.7372, 7.0911]","[50.7336, 7.095]",0.485359,4.853586
3,44717,bonn,00:07:00,2019-02-01 00:38:00,4,0,"[50.7476, 7.0826]","[50.7543, 7.0715]",1.079285,9.251016
4,44639,bonn,00:10:00,2019-02-01 00:44:00,4,0,"[50.7392, 7.0906]","[50.7525, 7.0712]",2.012538,12.075226
...,...,...,...,...,...,...,...,...,...,...
329624,44862,bonn,00:46:00,2019-06-30 23:49:00,6,23,"[50.7159, 7.1133]","[50.7253, 7.1535]",3.016851,3.935022
329625,44838,bonn,00:06:00,2019-06-30 23:50:00,6,23,"[50.6495, 7.2006]","[50.6577, 7.1942]",1.017326,10.173256
329626,45084,bonn,00:21:00,2019-06-30 23:54:00,6,23,"[50.7377, 7.1018]","[50.7096, 7.1155]",3.270024,9.342925
329627,45010,bonn,00:05:00,2019-06-30 23:55:00,6,23,"[50.7374, 7.1033]","[50.7444, 7.1004]",0.804671,9.656054


In [20]:
# Print all trips with a speed of more than 25 km/h
df_bonn_fast_travel = df_bonn[(df_bonn["km/h"] > 25)]
df_bonn_fast_travel.sort_values(["km/h"], ascending=False).drop(["day", "time", "orig_lat", "orig_lng", "dest_lat", "dest_lng"], axis=1)

,b_number,city,trip_duration,timestamp,weekday,hour,orig,dest,distance (km),km/h
253132,45198,bonn,00:03:00,2019-06-07 14:12:00,4,14,"[50.7106, 7.1268]","[50.8545, 6.8702]",24.113875,482.277505
134224,45239,bonn,00:03:00,2019-04-26 19:08:00,4,19,"[50.7286, 7.0944]","[50.893, 7.2478]",21.221347,424.426932
241687,44937,bonn,00:08:00,2019-06-04 14:07:00,1,14,"[50.8938, 6.9888]","[50.7044, 7.1753]",24.805968,186.044761
160325,44693,bonn,00:07:00,2019-05-07 16:07:00,1,16,"[50.7333, 7.0931]","[50.81, 6.9349]",14.017822,120.152762
38197,44646,bonn,00:04:00,2019-03-08 11:23:00,4,11,"[50.6939, 7.1795]","[50.7108, 7.0694]",7.978295,119.674425
...,...,...,...,...,...,...,...,...,...,...
55418,44989,bonn,00:11:00,2019-03-22 19:30:00,4,19,"[50.7057, 7.1323]","[50.6721, 7.1729]",4.705205,25.664753
202391,45427,bonn,00:17:00,2019-05-22 21:29:00,2,21,"[50.6836, 7.1598]","[50.7319, 7.0903]",7.266074,25.644969
16248,44764,bonn,00:10:00,2019-02-18 15:15:00,0,15,"[50.7012, 7.1373]","[50.7316, 7.1006]",4.254742,25.528454
185736,44792,bonn,00:07:00,2019-05-16 16:00:00,3,16,"[50.7068, 7.1123]","[50.7322, 7.0992]",2.971110,25.466661


In [21]:
# Get all entries which are slower than 25 km/h
df_bonn = df_bonn[~df_bonn.isin(df_bonn_fast_travel).all(1)]
df_bonn.drop(["day", "time", "orig_lat", "orig_lng", "dest_lat", "dest_lng"], axis=1)

,b_number,city,trip_duration,timestamp,weekday,hour,orig,dest,distance (km),km/h
0,44894,bonn,00:07:00,2019-02-01 00:03:00,4,0,"[50.7366, 7.1009]","[50.739, 7.093]",0.616674,5.285779
1,44973,bonn,00:20:00,2019-02-01 00:12:00,4,0,"[50.7364, 7.0927]","[50.732, 7.1004]",0.730093,2.190280
2,44802,bonn,00:06:00,2019-02-01 00:37:00,4,0,"[50.7372, 7.0911]","[50.7336, 7.095]",0.485359,4.853586
3,44717,bonn,00:07:00,2019-02-01 00:38:00,4,0,"[50.7476, 7.0826]","[50.7543, 7.0715]",1.079285,9.251016
4,44639,bonn,00:10:00,2019-02-01 00:44:00,4,0,"[50.7392, 7.0906]","[50.7525, 7.0712]",2.012538,12.075226
...,...,...,...,...,...,...,...,...,...,...
329624,44862,bonn,00:46:00,2019-06-30 23:49:00,6,23,"[50.7159, 7.1133]","[50.7253, 7.1535]",3.016851,3.935022
329625,44838,bonn,00:06:00,2019-06-30 23:50:00,6,23,"[50.6495, 7.2006]","[50.6577, 7.1942]",1.017326,10.173256
329626,45084,bonn,00:21:00,2019-06-30 23:54:00,6,23,"[50.7377, 7.1018]","[50.7096, 7.1155]",3.270024,9.342925
329627,45010,bonn,00:05:00,2019-06-30 23:55:00,6,23,"[50.7374, 7.1033]","[50.7444, 7.1004]",0.804671,9.656054


#### Essen:

In [22]:
# Get the position outliers
df_essen_bounds = df_essen[
    ((df_essen["orig"].str[0] > essen_bounds["north_east"][0]) | (df_essen["orig"].str[0] < essen_bounds["south_west"][0])) | 
    ((df_essen["orig"].str[1] > essen_bounds["north_east"][1]) | (df_essen["orig"].str[1] < essen_bounds["south_west"][1])) | 
    ((df_essen["dest"].str[0] > essen_bounds["north_east"][0]) | (df_essen["dest"].str[0] < essen_bounds["south_west"][0])) | 
    ((df_essen["dest"].str[1] > essen_bounds["north_east"][1]) | (df_essen["dest"].str[1] < essen_bounds["south_west"][1]))]

# Get all entries which are in the API bounds
df_essen = df_essen[~df_essen.isin(df_essen_bounds).all(1)]

In [23]:
# Plot zone for Essen
zone_map(essen_center, essen_bounds["south_west"], essen_bounds["north_east"])

In [24]:
# Print the outliers (but only relevant columns)
df_essen_bounds.drop(["day", "time", "orig_lat", "orig_lng", "dest_lat", "dest_lng"], axis=1)

,b_number,city,trip_duration,timestamp,weekday,hour,orig,dest
453,51572,essen,00:07:00,2019-02-11 15:30:00,0,15,"[51.438, 6.7647]","[51.4318, 6.7752]"
454,51582,essen,00:06:00,2019-02-11 15:36:00,0,15,"[51.4297, 6.7774]","[51.4283, 6.7857]"
460,51582,essen,00:05:00,2019-02-11 16:45:00,0,16,"[51.4283, 6.7858]","[51.4297, 6.7771]"
470,51582,essen,00:15:00,2019-02-11 19:36:00,0,19,"[51.4309, 6.777]","[51.4223, 6.7918]"
492,51572,essen,00:04:00,2019-02-12 05:49:00,1,5,"[51.4318, 6.7753]","[51.4351, 6.7695]"
503,51582,essen,00:05:00,2019-02-12 08:08:00,1,8,"[51.4224, 6.7918]","[51.4275, 6.7977]"
1589,51378,essen,00:16:00,2019-02-26 11:33:00,1,11,"[51.4331, 6.7951]","[51.431, 6.7658]"
1602,51329,essen,01:01:00,2019-02-26 12:47:00,1,12,"[51.4618, 7.0078]","[51.4486, 6.8525]"
1643,51378,essen,00:15:00,2019-02-26 18:14:00,1,18,"[51.4309, 6.7653]","[51.4278, 6.786]"
1650,51378,essen,00:09:00,2019-02-26 18:51:00,1,18,"[51.4278, 6.786]","[51.4395, 6.7829]"


In [25]:
# Print all entries which are in the API bounds (but only relevant columns)
df_essen.drop(["day", "time", "orig_lat", "orig_lng", "dest_lat", "dest_lng"], axis=1)

,b_number,city,trip_duration,timestamp,weekday,hour,orig,dest
0,50182,essen,00:18:00,2019-02-01 00:05:00,4,0,"[51.443, 7.0163]","[51.4439, 6.9778]"
1,53528,essen,00:08:00,2019-02-01 00:46:00,4,0,"[51.434, 7.0036]","[51.4267, 7.0028]"
2,52115,essen,00:09:00,2019-02-01 04:50:00,4,4,"[51.4618, 7.0078]","[51.4522, 7.0146]"
3,52196,essen,00:08:00,2019-02-01 05:41:00,4,5,"[51.4382, 7.0057]","[51.4328, 6.9888]"
4,53700,essen,00:10:00,2019-02-01 06:16:00,4,6,"[51.4544, 7.0113]","[51.4581, 7.0003]"
...,...,...,...,...,...,...,...,...
16693,52969,essen,00:14:00,2019-06-30 21:22:00,6,21,"[51.4581, 7.0003]","[51.4471, 6.9784]"
16694,50860,essen,00:08:00,2019-06-30 21:30:00,6,21,"[51.4416, 6.997]","[51.4439, 6.9778]"
16695,51210,essen,00:07:00,2019-06-30 21:37:00,6,21,"[51.4544, 7.0113]","[51.4579, 7.0167]"
16696,53187,essen,00:08:00,2019-06-30 22:00:00,6,22,"[51.4505, 7.0124]","[51.4636, 7.0097]"


In [26]:
# Plot outliers for Essen
outliers_zone_map(essen_center, essen_bounds["south_west"], essen_bounds["north_east"], df_essen_bounds)

Furthermore we want to remove trips with a high speed value.  
Therefore we calculate the linear distance between origin and destination with the haversine formular.  
Afterwards the km/h for each trip gets computed and journeys with a speed of more then 25 km/h are removed. 

In [27]:
# Calculate the distance in kilometers between origin and destination
df_essen["distance (km)"] = df_essen.apply(lambda row: haversine(row["orig"], row["dest"]), axis=1)
df_essen.drop(["day", "time", "orig_lat", "orig_lng", "dest_lat", "dest_lng"], axis=1)

,b_number,city,trip_duration,timestamp,weekday,hour,orig,dest,distance (km)
0,50182,essen,00:18:00,2019-02-01 00:05:00,4,0,"[51.443, 7.0163]","[51.4439, 6.9778]",2.670173
1,53528,essen,00:08:00,2019-02-01 00:46:00,4,0,"[51.434, 7.0036]","[51.4267, 7.0028]",0.813617
2,52115,essen,00:09:00,2019-02-01 04:50:00,4,4,"[51.4618, 7.0078]","[51.4522, 7.0146]",1.166822
3,52196,essen,00:08:00,2019-02-01 05:41:00,4,5,"[51.4382, 7.0057]","[51.4328, 6.9888]",1.316402
4,53700,essen,00:10:00,2019-02-01 06:16:00,4,6,"[51.4544, 7.0113]","[51.4581, 7.0003]",0.866113
...,...,...,...,...,...,...,...,...,...
16693,52969,essen,00:14:00,2019-06-30 21:22:00,6,21,"[51.4581, 7.0003]","[51.4471, 6.9784]",1.949080
16694,50860,essen,00:08:00,2019-06-30 21:30:00,6,21,"[51.4416, 6.997]","[51.4439, 6.9778]",1.355057
16695,51210,essen,00:07:00,2019-06-30 21:37:00,6,21,"[51.4544, 7.0113]","[51.4579, 7.0167]",0.539863
16696,53187,essen,00:08:00,2019-06-30 22:00:00,6,22,"[51.4505, 7.0124]","[51.4636, 7.0097]",1.468619


In [28]:
# Calculate the km/h for each trip
df_essen["km/h"] = df_essen.apply(lambda row: row["distance (km)"]/(row["trip_duration"].seconds/3600), axis=1)
df_essen.drop(["day", "time", "orig_lat", "orig_lng", "dest_lat", "dest_lng"], axis=1)

,b_number,city,trip_duration,timestamp,weekday,hour,orig,dest,distance (km),km/h
0,50182,essen,00:18:00,2019-02-01 00:05:00,4,0,"[51.443, 7.0163]","[51.4439, 6.9778]",2.670173,8.900577
1,53528,essen,00:08:00,2019-02-01 00:46:00,4,0,"[51.434, 7.0036]","[51.4267, 7.0028]",0.813617,6.102124
2,52115,essen,00:09:00,2019-02-01 04:50:00,4,4,"[51.4618, 7.0078]","[51.4522, 7.0146]",1.166822,7.778816
3,52196,essen,00:08:00,2019-02-01 05:41:00,4,5,"[51.4382, 7.0057]","[51.4328, 6.9888]",1.316402,9.873018
4,53700,essen,00:10:00,2019-02-01 06:16:00,4,6,"[51.4544, 7.0113]","[51.4581, 7.0003]",0.866113,5.196676
...,...,...,...,...,...,...,...,...,...,...
16693,52969,essen,00:14:00,2019-06-30 21:22:00,6,21,"[51.4581, 7.0003]","[51.4471, 6.9784]",1.949080,8.353199
16694,50860,essen,00:08:00,2019-06-30 21:30:00,6,21,"[51.4416, 6.997]","[51.4439, 6.9778]",1.355057,10.162928
16695,51210,essen,00:07:00,2019-06-30 21:37:00,6,21,"[51.4544, 7.0113]","[51.4579, 7.0167]",0.539863,4.627396
16696,53187,essen,00:08:00,2019-06-30 22:00:00,6,22,"[51.4505, 7.0124]","[51.4636, 7.0097]",1.468619,11.014641


In [29]:
# Print all trips with a speed of more than 25 km/h
df_essen_fast_travel = df_essen[(df_essen["km/h"] > 25)]
df_essen_fast_travel.sort_values(["km/h"], ascending=False).drop(["day", "time", "orig_lat", "orig_lng", "dest_lat", "dest_lng"], axis=1)

,b_number,city,trip_duration,timestamp,weekday,hour,orig,dest,distance (km),km/h
13585,53086,essen,00:03:00,2019-06-15 06:12:00,5,6,"[51.4714, 6.9495]","[51.4594, 7.0233]",5.283620,105.672394
16656,52298,essen,00:03:00,2019-06-30 15:36:00,6,15,"[51.4964, 7.0085]","[51.4522, 7.0146]",4.932948,98.658954
7028,51177,essen,00:03:00,2019-05-01 03:13:00,2,3,"[51.4505, 7.0124]","[51.4198, 7.0245]",3.515223,70.304454
2557,53038,essen,00:04:00,2019-03-19 14:35:00,1,14,"[51.4544, 7.0113]","[51.4463, 7.0751]",4.511894,67.678406
15002,53483,essen,00:03:00,2019-06-22 17:10:00,5,17,"[51.4714, 6.9495]","[51.4495, 6.973]",2.929293,58.585853
1399,52262,essen,00:03:00,2019-02-24 10:12:00,6,10,"[51.4306, 6.999]","[51.4544, 7.0113]",2.780359,55.607171
13104,52425,essen,00:05:00,2019-06-12 20:13:00,2,20,"[51.4102, 6.9929]","[51.4495, 6.973]",4.582568,54.990814
2235,51313,essen,00:03:00,2019-03-08 09:27:00,4,9,"[51.4439, 6.9778]","[51.4522, 7.0146]",2.712084,54.241676
12303,51538,essen,00:03:00,2019-06-07 06:43:00,4,6,"[51.438, 6.9837]","[51.4522, 7.0146]",2.660664,53.213270
13763,51580,essen,00:03:00,2019-06-16 16:42:00,6,16,"[51.4495, 6.973]","[51.465, 7.0017]",2.631471,52.629417


In [30]:
# Get all entries which are slower than 25 km/h
df_essen = df_essen[~df_essen.isin(df_essen_fast_travel).all(1)]
df_essen.drop(["day", "time", "orig_lat", "orig_lng", "dest_lat", "dest_lng"], axis=1)

,b_number,city,trip_duration,timestamp,weekday,hour,orig,dest,distance (km),km/h
0,50182,essen,00:18:00,2019-02-01 00:05:00,4,0,"[51.443, 7.0163]","[51.4439, 6.9778]",2.670173,8.900577
1,53528,essen,00:08:00,2019-02-01 00:46:00,4,0,"[51.434, 7.0036]","[51.4267, 7.0028]",0.813617,6.102124
2,52115,essen,00:09:00,2019-02-01 04:50:00,4,4,"[51.4618, 7.0078]","[51.4522, 7.0146]",1.166822,7.778816
3,52196,essen,00:08:00,2019-02-01 05:41:00,4,5,"[51.4382, 7.0057]","[51.4328, 6.9888]",1.316402,9.873018
4,53700,essen,00:10:00,2019-02-01 06:16:00,4,6,"[51.4544, 7.0113]","[51.4581, 7.0003]",0.866113,5.196676
...,...,...,...,...,...,...,...,...,...,...
16693,52969,essen,00:14:00,2019-06-30 21:22:00,6,21,"[51.4581, 7.0003]","[51.4471, 6.9784]",1.949080,8.353199
16694,50860,essen,00:08:00,2019-06-30 21:30:00,6,21,"[51.4416, 6.997]","[51.4439, 6.9778]",1.355057,10.162928
16695,51210,essen,00:07:00,2019-06-30 21:37:00,6,21,"[51.4544, 7.0113]","[51.4579, 7.0167]",0.539863,4.627396
16696,53187,essen,00:08:00,2019-06-30 22:00:00,6,22,"[51.4505, 7.0124]","[51.4636, 7.0097]",1.468619,11.014641


---

### Drop unnecessary columns

Now that outliers are removed, we don't need the columns "distance" and "km/h" anymore.  
We also don't need the individual columns "orig_lat", "orig_lng", "dest_lat" and "dest_lng" anymore.  

In [31]:
df_bonn = df_bonn.drop(["orig_lat", "orig_lng", "dest_lat", "dest_lng", "distance (km)", "km/h"], axis=1)
df_essen = df_essen.drop(["orig_lat", "orig_lng", "dest_lat", "dest_lng", "distance (km)", "km/h"], axis=1)

From here on, work with outlier-free data. 

---

### Gap in the data?

We search for huge gaps (> 1 day) in the data:

#### Bonn:

In [32]:
# Sort by timestamp, if not already, just to be safe, then reset the index
df_bonn = df_bonn.sort_values(["timestamp"])
df_bonn = df_bonn.reset_index().drop(["index"], axis=1)

In [33]:
# Which timedelta between two rows is maximal? (i.e. no data between two points in time)
# df_bonn.drop(0) drops 0th row (so we can subtract a row's timestamp and the next row's timestamp).
# df_bonn.drop(0)-df_bonn simply means we calculate df_bonn.iloc[i+1]-df_bonn.iloc[i] for all i.
# (of course, only "timestamp" column). This corresponds to the time difference between two rows. 
# We then print the row whose timedelta to the next row is maximal.
# By the way, we need to reset the index of df_bonn.drop(0). Otherwise, it will subtract the wrong rows since pandas
# uses indexes to match rows. 

print("Biggest gap:")
td = (df_bonn.drop(0).reset_index()["timestamp"]-df_bonn["timestamp"]) # timedeltas
td1, td2 = df_bonn.iloc[td.idxmax()]["timestamp"], df_bonn.iloc[td.idxmax()+1]["timestamp"]
print(td.max()) # duration of the gap
print(td1) # start of the gap
print(td2) # end of the gap
print("\nNumber of huge gaps (> 1 day):", td[td>timedelta(days=1)].count())

Biggest gap:
3 days 20:24:00
2019-03-15 13:05:00
2019-03-19 09:29:00

Number of huge gaps (> 1 day): 1


#### Essen:

In [34]:
# Sort by timestamp, if not already, just to be safe, then reset the index
df_essen = df_essen.sort_values(["timestamp"])
df_essen = df_essen.reset_index().drop(["index"], axis=1)

In [35]:
# Calculating the gaps like mentioned before

print("Biggest gap:")
td = (df_essen.drop(0).reset_index()["timestamp"]-df_essen["timestamp"]) # timedeltas
td1, td2 = df_essen.iloc[td.idxmax()]["timestamp"], df_essen.iloc[td.idxmax()+1]["timestamp"]
print(td.max()) # duration of the gap
print(td1) # start of the gap
print(td2) # end of the gap
print("\nNumber of huge gaps (> 1 day):", td[td>timedelta(days=1)].count())

Biggest gap:
3 days 23:12:00
2019-03-15 10:18:00
2019-03-19 09:30:00

Number of huge gaps (> 1 day): 1


It seems like there has been an issue while collecting the data between 2019-03-15 and 2019-03-19.    
We will keep this in mind for further data visualization.

---

### Export the data

In [36]:
# Save to pickle files so that we don't have to run Preprocessing.ipynb every single time we start a new kernel.
df_bonn.to_pickle("data/df_bonn.pickle")
df_essen.to_pickle("data/df_essen.pickle")